In [ ]:
import requests
import re
import os

In [ ]:
# judge if the folder exist. 判断图集的文件夹是否存在，如果不存在则建立一个
def fJudgeFileCOILS(vSeqName):    
    global vFoldNameCOILS
    vFoldNameCOILS = "E:\\Store\\code\\02.Python\\03.Request\\Bio" + "\\" + vSeqName + "\\COILS\\"
    os.makedirs(vFoldNameCOILS, exist_ok = True)
    os.chdir(vFoldNameCOILS)
    os.makedirs("Pic", exist_ok = True)
    os.makedirs("Text", exist_ok = True)

In [ ]:
def fGetCOILS(vSeq):
    vUrl = "https://embnet.vital-it.ch/cgi-bin/COILS_form_parser"
    vPostData ={"win": "all",
                "mat": "MTIDK",
                "weight": "no",
                "comm": "",
                "format": "plain_text",
                "seq": vSeq}
    r = requests.post(vUrl, data = vPostData)
    return(r.text)

In [ ]:
def fSaveCOILSPic(vHtml):
    global vCode    
    vCode = str(re.search("/download.php\?file=(.*?).gif", vHtml, re.S).group(1))    
    vPicA = "https://embnet.vital-it.ch/download.php?file=" + vCode + ".gif"    
    vPicB = "https://embnet.vital-it.ch/download.php?file=" + vCode + ".ps"
    vTxtA = "https://embnet.vital-it.ch/download.php?file=" + vCode + ".out"    
    with open("Pic\\COILS.gif", "wb") as vPic1:
        r = requests.get(vPicA)
        vPic1.write(r.content)
    with open("Pic\\COILS.ps", "wb") as vPic2:
        r = requests.get(vPicB)
        vPic2.write(r.content)
    with open("Text\\COILS.txt", "wb") as vTxt1:
        r = requests.get(vTxtA)
        vTxt1.write(r.content)    

In [ ]:
def fSoupCOILS(vHtml):
    vCOILSText = vHtml
    vCOILS1 = vCOILSText.replace(re.search("[\s\S]*\</h1\>", vCOILSText, re.S).group(0), "<h1>COILS卷曲螺旋分析</h1><p><a href = \"https://embnet.vital-it.ch/software/COILS_form.html\">使用COILS5.0查询</a></p>")
    vCOILS2 = vCOILS1.replace(re.search("\<div id='sib_footer'\>[\s\S]*", vCOILS1, re.S).group(0), "").replace("GIF-format", "gif图片")
    vCOILS3 = vCOILS2.replace("Date:", "日期：").replace("Postscript-format", "PS格式图片")
    vCOILS4 = vCOILS3.replace("/download.php?file=" + vCode + ".gif", "file:///" + vFoldNameCOILS + "\Pic" + "\COILS.gif")
    vCOILS5 = vCOILS4.replace("/download.php?file=" + vCode + ".ps", "file:///" + vFoldNameCOILS + "\Pic" + "\COILS.ps")
    vCOILS6 = vCOILS5.replace("/download.php?file=" + vCode + ".out", "file:///" + vFoldNameCOILS + "\Text" + "\COILS.txt")
    vCOILS7 = vCOILS6.replace("numerical format (window 14, 21, 28)", "预测数据（以14、21、28为循环个数）")
    vCOILS8 = vCOILS7.replace("You can get the prediction graphics shown above in one of", "")
    vCOILS9 = vCOILS8.replace("he following formats:", "你可以下载以下类型的预测数据：").replace("using ", "使用")
    vCOILS10 = vCOILS9.replace(" matrix", "矩阵").replace("Input file is ", "本次查询的工作编号：").replace("NCOILS version", "NCOILS版本").replace("no weights", "未预测到卷曲螺旋结构")
    vCOILS = vCOILS10
    with open("COILS.html", "w") as f:
        f.write(vCOILS)

In [ ]:
def main(vSeqName, vSeq):
    print("=====开始COILS跨膜结构域分析====")
    fJudgeFileCOILS(vSeqName)
    print("---------已创建存储目录---------")
    print("----------开始提交数据----------")
    vHtml = fGetCOILS(vSeq)
    print("---------已获取分析结果---------")
    print("----------开始保存数据----------")
    fSaveCOILSPic(vHtml)
    print("----------开始保存报告----------")
    fSoupCOILS(vHtml)
    print("---------结果已保存完毕---------")

In [ ]:
with open("name.txt", "r") as f:
    vSeqName = f.read()
with open("orf.txt", "r") as f:
    vSeq = f.read()
main(vSeqName, vSeq)